## Tokenization using Byte-Pair Encoding and a Unigram Language Model

Author: Pierre Nugues with help from Marcus Klang

In this assignment, you will create a tokenization program to handle subwords.

In many scripts from Asia, like Chinese, Korean, or Japanese scripts, tokenization cannot rely on white spaces. The byte-pair encoding and the unigram language model are techniques that are now common in machine translation to carry out a tokenization at a subword level. Subword level tokenization shows better multilingual capabilities.

You will follow two papers: 
* Subword Regularization: _Improving Neural Network Translation Models with Multiple Subword Candidates_ by Kudo (2018) (https://arxiv.org/pdf/1804.10959.pdf) and 
* _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020) (https://aclanthology.org/2020.findings-emnlp.414.pdf). 

In addition, you will start from a clear and easy-to-understand description in Google’s Neural Machine Translation System: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016). (Do not read them now)
https://arxiv.org/abs/1609.08144

You will use a small corpus make it easier to test and correct your code. Note also that you will use _characters_ and not _bytes_ in this lab as this is simpler to implement. For a complete program, see the link at the end.

**In your report, be sure to answer all the questions. Please reuse the section titles of this notebook so that I can check your answers more easily**

## Preliminaries

As an overall description of the subword tokenizers, read Sections 4 (introduction paragraph) and 4.1. in the paper on translation: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016), https://arxiv.org/abs/1609.08144.  

In your report, in a few lines (10 to 15 lines or so) you will:

1. Outline the difference with tokenization as you saw it during the course;
2. Imagine how the tokens will be learned (this will developed in the rest of the lab);
3. Summarize what could be the advantages for Asian languages, unknown words, and translation.

Commenting Sections 4 and 4.1 in your report is **mandatory**. If you are curious, you can read the complete article.

## Understanding SentencePiece
SentencePiece is the combination of BPE and a language model. To be sure you understand how it works, you will first run this code.
These are the first cells from a larger program: https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb
from Google.


In [ ]:
%pip install sentencepiece
!wget https: // raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt


In [1]:
import sentencepiece as spm


Train a model from a corpus. Once trained, read the content of `m.vocab`. Be sure to undestand its content.

In [2]:
spm.SentencePieceTrainer.train(
    '--input=botchan.txt --model_prefix=m --vocab_size=2000')


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=botchan.txt --model_prefix=m --vocab_size=2000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: botchan.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  p

In [3]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')


True

In [4]:
print(sp.encode('Tokenization using Byte-Pair Encoding'))


[386, 315, 27, 52, 1997, 227, 282, 14, 1058, 237, 22, 717, 1060, 443, 27, 67, 38, 20, 14]


In [5]:
sp.id_to_piece(386)


'▁To'

In [6]:
[sp.id_to_piece(i) for i in range(20)]


['<unk>',
 '<s>',
 '</s>',
 ',',
 '.',
 '▁the',
 '▁I',
 's',
 '▁to',
 '▁a',
 '▁and',
 '▁of',
 '▁',
 'ed',
 'ing',
 '▁in',
 '▁was',
 '▁"',
 '▁it',
 't']

In [7]:
print(sp.encode('Tokenization using Byte-Pair Encoding', out_type=str))


['▁To', 'ke', 'n', 'i', 'z', 'ation', '▁us', 'ing', '▁By', 'te', '-', 'P', 'air', '▁E', 'n', 'c', 'o', 'd', 'ing']


## Design of the BPE Algorithm

The first algorithm to build the subwords from a corpus is a byte-pair encoding (BPE), due to Gage (1994). In the lab, you will first read two sections of more recent articles as they are easier to understand and specifically targeted to natural language processing.

Read these two sections:

1. Section 3.1 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 1 of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the byte-pair encoding (BPE) algorithm as described by Kudo (2018) and Bostrom and Durrett (2020) (Only BPE and not the unigram language model).

## BPE Programming

You will now program a byte-pair encoding program in Python. You will do it step by step. The first part will be to extract the subwords from a corpus. Note that you will use the characters, not the bytes.

In [7]:
import regex as re
import tqdm as tqdm
import math


First use a small corpus and then, if you have time, test your program on a larger one. Here we take the smallest novel from Selma Lagerlöf in our corpus.

In [8]:
import os
from zipfile import ZipFile
import requests

# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname)
    for fname in
    [
        "bannlyst.txt",
        "gosta.txt",
        "herrgard.txt",
        "jerusalem.txt",
        "kejsaren.txt",
        "marbacka.txt",
        "nils.txt",
        "osynliga.txt",
        "troll.txt"
    ]
]


def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""

    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()

        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith(
            "__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)

        print("Done!")


# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")

SELMA_FILES


Selma has been downloaded.


['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

In [9]:
#FILE_PATH = '../../corpus/Selma.txt'
FILE_PATH = 'Selma/herrgard.txt'


Read the corpus and store it in the `corpus` string variable.

In [10]:
with open(FILE_PATH, encoding='utf8') as f:
    corpus = f.read().strip()


Replace all the space sequences in `corpus`, including newlines and tabulations, and normalize them as one space. Use the `\s` class.

In [11]:
# Write your code



In [12]:
corpus[:100]


'Selma Lagerlöf En herrgårdssägen Bokutgåva Albert Bonniers förlag, Stockholm 1899. I. Det var en skö'

### BPE

#### Initial Vocabulary

Write the code (one instruction) to split the corpus in a list of characters and store the results in `corpus_l`. This is just a type conversion. Given the input:
<pre><span style="font-size: 12pt;">corpus = 'De senaste fem &aring;ren har cirka 25 000 unga'</span></pre>

Return:
<pre><span style="font-size: 12pt;">corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', 'e', ' ', 'f', 'e', 'm', ' ', ...]</span></pre>

In [13]:
# Write your code
)


In [14]:
corpus_l[:15]


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'a', 'g', 'e', 'r', 'l', 'ö', 'f', ' ']

Extract the set of characters that will serve as initial subword tokens:

1. Write a statement to extract the set of all the characters from `corpus_l`; 
2. Exclude the space from this set and call the resulting set: `char_set`.

In [15]:
# Write your code



In [19]:
print(sorted(char_set))

['!', ',', '-', '.', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '»', 'Ä', 'Å', 'Ö', 'ä', 'å', 'é', 'ö', '–', '’']


In [20]:
len(char_set)


67

Using code from the previous question, write an `initial_vocabulary()` function taking the the `corpus_l` variable as input and returning the the set of all characters appearing in the corpus (the initial character set), deprived from the white space.

In [21]:
# Write your code here
def initial_vocabulary(corpus_l):
    ...
    return vocabulary


In [19]:
initial_vocabulary(corpus_l)


{'!',
 ',',
 '-',
 '.',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'å',
 'é',
 'ö',
 '–',
 '’'}

#### Counting

Write a `pair_count()` function that takes a list of tokens as input, possibly single characters or subword tokens, and that counts the adjacent pairs (bigrams). You will implement these counts as dictionaries: The key will be a pair (tuple) of adjacent symbols and the value, its frequency. Remember that you cannot cross whitespaces, i.e. a pair cannot include a whitespace.

Given the input

`['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`
count_pairs should return a dictionary: 


`{('D', 'e'): 1, ('s', 'e'): 1, ('e', 'n'): 1, ('n', 'a'): 1, ...}`

In [22]:
# Write your code here
def pair_count(corpus_l):
    pairs = {}
    ...
    return pairs


In [23]:
pairs = pair_count(corpus_l)


In [22]:
pairs


{('S', 'e'): 7,
 ('e', 'l'): 511,
 ('l', 'm'): 22,
 ('m', 'a'): 424,
 ('L', 'a'): 8,
 ('a', 'g'): 477,
 ('g', 'e'): 646,
 ('e', 'r'): 1281,
 ('r', 'l'): 213,
 ('l', 'ö'): 70,
 ('ö', 'f'): 217,
 ('E', 'n'): 21,
 ('h', 'e'): 773,
 ('r', 'r'): 177,
 ('r', 'g'): 140,
 ('g', 'å'): 327,
 ('å', 'r'): 295,
 ('r', 'd'): 413,
 ('d', 's'): 83,
 ('s', 's'): 218,
 ('s', 'ä'): 205,
 ('ä', 'g'): 143,
 ('e', 'n'): 3439,
 ('B', 'o'): 2,
 ('o', 'k'): 54,
 ('k', 'u'): 562,
 ('u', 't'): 295,
 ('t', 'g'): 43,
 ('å', 'v'): 2,
 ('v', 'a'): 1426,
 ('A', 'l'): 24,
 ('l', 'b'): 44,
 ('b', 'e'): 314,
 ('r', 't'): 381,
 ('o', 'n'): 1600,
 ('n', 'n'): 1070,
 ('n', 'i'): 231,
 ('i', 'e'): 22,
 ('r', 's'): 258,
 ('f', 'ö'): 980,
 ('ö', 'r'): 1317,
 ('l', 'a'): 873,
 ('g', ','): 192,
 ('S', 't'): 88,
 ('t', 'o'): 351,
 ('o', 'c'): 1223,
 ('c', 'k'): 794,
 ('k', 'h'): 23,
 ('h', 'o'): 1098,
 ('o', 'l'): 211,
 ('1', '8'): 1,
 ('8', '9'): 1,
 ('9', '9'): 1,
 ('9', '.'): 1,
 ('I', '.'): 6,
 ('D', 'e'): 429,
 ('e', 't'): 

Determine the most frequent pair

In [24]:
# write your code
most_freq_pair = 


In [25]:
most_freq_pair


('d', 'e')

In [26]:
''.join(most_freq_pair)


'de'

#### The First Iteration

We store the initial symbols in a `vocabulary` variable

In [27]:
vocabulary = initial_vocabulary(corpus_l)


In [28]:
len(vocabulary)


67

Add your most frequent pair to the vocabulary after one iteration as well as the merge operations: `merge_ops`. `merge_ops` will contain the merge operations in the order you created them.

In [29]:
merge_ops = []


In [30]:
# write your code here



In [31]:
len(vocabulary)


68

In [32]:
merge_ops


[('d', 'e')]

#### Incremental Construction
We will now incrementally build the vocabulary.

Create a `merge_bigrams()` function that takes a list of tokens, `corpus_l`, and a pair of subword tokens `(token_r, token_l)` as input and merges adjacent sequences token_r, token_l into a new token, `token_new`, replacing the sequence `token_r, token_l` in `corpus_l`. Your function will return a new list. 

Given the input 

`corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`

`merge_bigrams(corpus_l, ('e', 'n'))` should return where all the seuquences of 'e' and 'n' have been merged:

`['D', 'e', ' ', 's', 'en', 'a', 's', 't', ...]`

And reapplying `merge_bigrams(corpus_l, ('s', 'en'))` to this corpus should return

`['D', 'e', ' ', 'sen', 'a', 's', 't', ...]`

You will apply a greedy algorithm. Given the pair ('a', 'a') and the list ['a', 'a', 'a'], the result will be: ['aa', 'a']

In [33]:
# Write your code here
def merge_bigrams(corpus_l, pair):
    new_corpus_l = []
    ...
    return new_corpus_l


In [34]:
corpus_test = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't']
merge_bigrams(corpus_test, ('e', 'n'))


['D', 'e', ' ', 's', 'en', 'a', 's', 't']

In [35]:
merge_bigrams(merge_bigrams(corpus_test, ('e', 'n')), ('s', 'en'))


['D', 'e', ' ', 'sen', 'a', 's', 't']

#### Byte Pair Encoding (BPE): Building the Vocabulary

Write now a `BPE()` function following Algorithm 1 in _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020). 

Your function will take `corpus_l` and the vocabulary size `k` as input. This size `k` will correspond to the count of new subwords added to the initial list of symbols. With your initial corpus, you should have 67 found symbols. With `k = 10`, you will add 10 subwords to this initial list. Note that Bostrom and Durrett (2020) define their $k_\text{Bostrom and Durrett}$ as `k + initial vocabulary`. 

Return the vocabulary of subword tokens in the form of a list, the initial vocabulary and the subwords you will create, as well as the merge operations in the order you created them

You will start from the initial vocabulary and `k` will be the number of symbols you add to this vocabulary.

In [36]:
# Write your code here
def BPE(corpus_l, k):
    vocabulary = initial_vocabulary(corpus_l)
    merge_ops = []
    ...
    return vocabulary, merge_ops


We build a vocabulary of 50 subwords in addition to our initial set of symbols

In [37]:
vocabulary, merge_ops = BPE(corpus_l, 50)


de en an tt ar st om on ll ör att ch ade ig er ng och var hon et för sk är ck han or na det ne så än in ej un ill den som fv på ed ag li enne henne id ra hade all ing ta 

In [38]:
len(vocabulary), len(merge_ops)


(117, 50)

In [40]:
print(vocabulary)


{'T', 'ar', 'D', 'll', 'han', 'I', 'li', 'e', '9', 'n', 'ra', 'id', 'ng', 'in', 'så', 'all', 'V', 't', 'ta', 'fv', 'F', 'som', '–', 'A', 'ö', 'u', 'för', 'ör', 'C', 'un', 'Ä', 'ck', 'et', 'R', 'j', '’', 'ch', 'h', 'tt', 'U', 'om', 'y', 'on', ';', 'Ö', 'or', 'ill', 'på', 'Å', 'r', 'K', 'z', 'P', 'an', 'var', 'o', 'k', ',', 'f', 'ade', 'g', 'l', 'och', 'G', '8', 'J', 'L', 'än', 'henne', 'E', '_', 'enne', 'O', 'i', 'B', 'det', 'ag', 's', '-', 'är', 'ä', 'v', 'N', '!', 'p', '?', 'ne', 'er', 'a', 'é', 'att', 'c', 'sk', 'X', 'de', 'm', 'M', 'na', 'H', 'hon', 'en', 'd', 'b', 'å', 'S', '.', 'ig', 'ej', '»', 'ing', '1', 'st', 'x', 'hade', ':', 'ed', 'den'}


In [41]:
print(merge_ops)


[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('c', 'h'), ('a', 'de'), ('i', 'g'), ('e', 'r'), ('n', 'g'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('n', 'e'), ('s', 'å'), ('ä', 'n'), ('i', 'n'), ('e', 'j'), ('u', 'n'), ('i', 'll'), ('de', 'n'), ('s', 'om'), ('f', 'v'), ('p', 'å'), ('e', 'd'), ('a', 'g'), ('l', 'i'), ('en', 'ne'), ('h', 'enne'), ('i', 'd'), ('r', 'a'), ('h', 'ade'), ('a', 'll'), ('i', 'ng'), ('t', 'a')]


#### BPE Tokenizer

You will now use the vocabulary you obtained to tokenize a text stored in the corpus string.

You will apply the merge operations in the same order you created them. You will call this function `tokenize_bpe()` that will take two inputs: `corpus` and `merge_ops`, and that will return the tokenized text in the form of a list.

    def tokenize_bpe(corpus, merge_ops):

      ...

      return tokens

This function is easy. Just reuse the `merge_bigrams()` function and a loop

In [42]:
# Write your code here
def tokenize_bpe(corpus, merge_ops):
    corpus_l = list(corpus)
    ...
    return corpus_l


In [43]:
print(tokenize_bpe(corpus, merge_ops)[:200])


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'ag', 'er', 'l', 'ö', 'f', ' ', 'E', 'n', ' ', 'h', 'er', 'r', 'g', 'å', 'r', 'd', 's', 's', 'ä', 'g', 'en', ' ', 'B', 'o', 'k', 'u', 't', 'g', 'å', 'v', 'a', ' ', 'A', 'l', 'b', 'er', 't', ' ', 'B', 'on', 'n', 'i', 'er', 's', ' ', 'för', 'l', 'ag', ',', ' ', 'S', 't', 'o', 'ck', 'h', 'o', 'l', 'm', ' ', '1', '8', '9', '9', '.', ' ', 'I', '.', ' ', 'D', 'et', ' ', 'var', ' ', 'en', ' ', 'sk', 'ö', 'n', ' ', 'h', 'ö', 'st', 'd', 'ag', ' ', 'h', 'ä', 'ne', 'm', 'o', 't', ' ', 's', 'l', 'u', 't', 'et', ' ', 'a', 'f', ' ', 't', 'r', 'e', 'tt', 'i', 'o', 'ta', 'l', 'et', '.', ' ', 'P', 'å', ' ', 'den', ' ', 't', 'i', 'den', ' ', 'f', 'an', 'n', 's', ' ', 'i', ' ', 'U', 'p', 's', 'a', 'l', 'a', ' ', 'e', 'tt', ' ', 'h', 'ö', 'g', 't', ',', ' ', 'g', 'u', 'l', 't', ' ', 't', 'v', 'å', 'v', 'å', 'n', 'ing', 's', 'h', 'u', 's', ',', ' ', 'som', ' ', 'st', 'o', 'd', ' ', 'un', 'de', 'r', 'l', 'ig', 't', ' ', 'en', 's', 'a', 'm', 't', ' ', 'på', ' ', 'en', ' ',

## Unigram Language Model

You are now done with BPE and you can now consider the unigram language model.

Read these two sections:

1. Section 3.2 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 2 and the related text of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the tokenization with a unigram language model as described by by Kudo (2018) and Bostrom and Durrett (2020). You will notably consider two aspects:
1. How to obtain the subword vocabulary;
2. How to tokenize a text.

In your report, given what you have done on the byte-pair encoding, how would you build the “reasonably big seed vocabulary” needed for the unigram language model?

### Unigram Probabilities

Starting from the “reasonably big seed vocabulary”, you will now fit a unigram language model. You will start with a vocabulary of 50 subwords in addition to the character set and reduce it to 49, i.e. you will find one subword to discard.

Kudo (2018) proposes the expectation-maximization algorithm. We will ignore this step. Instead, in this lab, you will approximate the language model with the BPE algorithm.

Write a `unigram_lm()` function that takes a corpus string and a vocabulary of subword tokens as input and returns a dictionary, where the keys are the subwords and each key value, the key relative frequency:

    def unigram_lm(corpus, vocabulary):

       ...

      return unigram_probs
Your function will:

1. Tokenize your corpus with BPE (you can reuse the `tokenize_bpe()` function);
2. Estimate the probability of each word (simply count the occurrences of the subwords and divide them by the length of the tokenized corpus);
3. Return this model as a dictionary.

In [44]:
# Write your code here
def unigram_lm(corpus, merge_ops):
    tokenized_corpus = tokenize_bpe(corpus, merge_ops)
    unigram_probs = {}
    cnt = 0
    ...
    return unigram_probs


In [45]:
unigram_probs = unigram_lm(corpus, merge_ops)
unigram_probs


{'S': 0.0017194185598387304,
 'e': 0.03049497514748461,
 'l': 0.03290611381760329,
 'm': 0.028795320019368163,
 'a': 0.040515035030682725,
 'L': 0.0003755052027234009,
 'ag': 0.004713578465764795,
 'er': 0.010830360583811774,
 'ö': 0.010069468462503828,
 'f': 0.021018409636649308,
 'E': 0.0002668063282508375,
 'n': 0.017569690801110704,
 'h': 0.015751455082660553,
 'r': 0.030929770645374863,
 'g': 0.024684526221133037,
 'å': 0.021225925669733293,
 'd': 0.029457394982064684,
 's': 0.03798531577023034,
 'ä': 0.015257369289603447,
 'en': 0.02391375238396395,
 'B': 0.0008103007006136546,
 'o': 0.015810745377827406,
 'k': 0.024486891903910193,
 'u': 0.02201646293862466,
 't': 0.0441910333310276,
 'v': 0.014605176042768066,
 'A': 0.0006225480992519541,
 'b': 0.015079498404102889,
 'on': 0.00571163176774015,
 'i': 0.020682431297370477,
 'för': 0.009061533444667333,
 ',': 0.027095664891251717,
 'ck': 0.00784608239374685,
 '1': 9.88171586114213e-06,
 '8': 9.88171586114213e-06,
 '9': 1.976343172

In [46]:
len(unigram_probs)


117

### Unigram Tokenization

You will now apply your unigram language model to tokenize a character sequence that does not include spaces, typically a single word in the Latin or Greek scripts or a sequence of words in Asian scripts, like Chinese or Korean.

Write a `tokenize_lm()` function that takes a character sequence, `char_seq`, and a dictionary of unigram probabilities, `unigram_probs`,  as input and returns the subword tokens and the segmentation probability, (prob,tokens). You will only return the token list with the highest probability.

    def tokenize_lm(char_seq, unigram_probs):

      ...

      return max(candidates)

As an example, applying 

`tokenize_lm('senare', unigram_probs)`
results in

`(2.0899522820189735e-07, ['s', 'en', 'ar', 'e'])`

Your function will cache (memoize) the results to speed up the computation. It will be similar to that of Norvig's in the notebook: How to Do Things with Words.ipynb. You can reuse it.
Python has a built-in memoization function that you can use: @functools.lru_cache(maxsize=2**10). You can also use the newer @functools.cache() function if you have Python 3.9 or higher. See here: https://docs.python.org/3/library/functools.html.

In [47]:
import functools


def tokenize_lm(char_seq, unigram_probs):
    # Use one of the two cache functions below to have a faster answer:
    # @functools.lru_cache(maxsize=2**10)
    @functools.cache  # Available from Python 3.9
    # The arguments of the cached function must be hashable that's why we define an inner cacheable function
    def __tokenize_lm(char_seq):
        # Write your code here
        ...

    return __tokenize_lm(char_seq)


In [48]:
tokenize_lm('senare', unigram_probs)


(2.5867587292712403e-07, ['s', 'en', 'ar', 'e'])

### Text Tokenization with Unigrams

The previous function applies to a sequence without spaces. You will now apply it to your corpus. Write a `tokenize_text_lm()` function that takes the whole `corpus` string as input and the unigram probabilities `unigram_probs` and return the corpus probability and the tokenized subwords. 

This function is just an application of the functions you just wrote, where you will:
1. `str.split()` the string by whitespaces
2. Break the tokens into subtokens using `tokenize_lm()`. This function will return the probabilities of the resulting sequences;
3. Sum the logarithm of these probabilities. Use log10 to check your output with the numbers in the notebook. 

It is very significant that you use the logarithm of the probabilities and the sum. If you multiply the probabilities, you will get an underflow.

In [49]:
# Write your code
def tokenize_text_lm(corpus, unigram_probs):
    tokenized_corpus = []
    corpus_prob = 0.0
    ...
    return corpus_prob, tokenized_corpus


In [50]:
init_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)


In [51]:
init_loglikelihood, tokens[:10]


(-183394.0764900353, ['▁S', 'e', 'l', 'm', 'a', '▁L', 'ag', 'er', 'l', 'ö'])

### Vocabulary Selection

You will now implement the final loop, where you will, at each iteration:
1. Select one subword and remove it from your vocabulary.
2. Estimate the probabilities of the subwords of the reduced vocabulary using `unigram_lm()`
2. Compute the resulting log-likelihood of the corpus without this word. You will use `tokenize_text_lm()` this time
3. Compute the loss, i.e. the log-likelihood reduction when the subword is removed from the current vocabulary

You will always keep the single characters in your vocabulary to avoid unknown words.

Store the pairs, (log-likelihood, removed_subword) in a list `logloss_word` and rank them by likelihood value.

In [52]:
logloss_word = []


In [53]:
# To have the same results
vocabulary_sorted = sorted(vocabulary, key=lambda w: -unigram_probs[w])
vocabulary_sorted[:10]


['t', 'a', 's', 'l', 'r', 'e', 'd', 'm', ',', 'g']

In [54]:
# Write your code here
for i, word in enumerate(tqdm.tqdm(vocabulary_sorted)):
    if len(word) == 1:
        continue
    ...


  0%|          | 0/117 [00:00<?, ?it/s]

100%|██████████| 117/117 [05:25<00:00,  2.78s/it]


In [55]:
sorted(logloss_word, reverse=True)


[(-10524.100611375936, 'ta'),
 (-10635.548926578631, 'ra'),
 (-10660.05402475709, 'ag'),
 (-10673.84841250212, 'var'),
 (-10673.84841250212, 'som'),
 (-10673.84841250212, 'och'),
 (-10673.84841250212, 'ing'),
 (-10673.84841250212, 'ill'),
 (-10673.84841250212, 'hon'),
 (-10673.84841250212, 'henne'),
 (-10673.84841250212, 'han'),
 (-10673.84841250212, 'hade'),
 (-10673.84841250212, 'för'),
 (-10673.84841250212, 'enne'),
 (-10673.84841250212, 'det'),
 (-10673.84841250212, 'den'),
 (-10673.84841250212, 'att'),
 (-10673.84841250212, 'all'),
 (-10673.84841250212, 'ade'),
 (-10779.939897996781, 'id'),
 (-10786.685836769728, 'ed'),
 (-10801.597924158617, 'li'),
 (-10817.553130551387, 'ng'),
 (-10869.284547945834, 'na'),
 (-10873.334727177222, 'så'),
 (-10905.971702064562, 'fv'),
 (-10918.84405772877, 'et'),
 (-10976.091295432212, 'ne'),
 (-10985.80312557827, 'in'),
 (-11004.788105880085, 'or'),
 (-11007.224169128225, 'sk'),
 (-11033.673005600547, 'tt'),
 (-11053.38942833431, 'un'),
 (-11084.1

You will reduce now your vocabulary by one token: `out_candidate`. Write the piece of code to determine it.

In [56]:
# Write your code here
out_candidate = ...


In [57]:
out_candidate


'ta'

## The tokenization without the subword

In [58]:
vocabulary_sorted.remove('ta')

In [59]:
unigram_probs = unigram_lm(corpus, vocabulary_sorted)


In [60]:
new_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)


In [61]:
init_loglikelihood

-183394.0764900353

In [62]:
new_loglikelihood


-193918.17710141125

In [63]:
tokens[:10]


['▁S', 'e', 'l', 'm', 'a', '▁L', 'ag', 'er', 'l', 'ö']

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will write a report where you will:
1. Describe the background as well as the algorithms you used. For this, summarize the articles as described in the notebook:
   * Preliminaries: subword tokenizers
   * Design of the BPE Algorithm
   * Unigram Language Model
2. Describe your program as well as your results

The whole report should be of 2 to 3 pages.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 29, 2023.

## Curious?

If you are interested, you can read the sentencepiece implementation by the author:
https://github.com/google/sentencepiece/tree/master

Concerning BPE, you can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py